In [1]:
from datasets import load_dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer,TrainingArguments,Trainer
from transformers import default_data_collator
import os 
from datasets import DatasetDict
import torch
import time
import gc
import json
import collections
from tqdm.auto import tqdm
from evaluate import load
import numpy as np 
import wandb
from itertools import product
import torch
wandb.init(mode="disabled")

In [ ]:
file_to_train='strict_model_individual_datasets'
path='./SlovakBabyLM/Curricullum_learning/'

In [2]:
dataset = load_dataset("TUKE-DeutscheTelekom/squad-sk")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 124546
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11422
    })
})

In [4]:
def filter_empty_answers(dataset):
    return dataset.filter(lambda x: len(x['answers']['text']) > 0)

dataset['train']=filter_empty_answers(dataset['train'])
dataset['validation'] = filter_empty_answers(dataset['validation'])

In [5]:
test_size = int(0.1 * len(dataset['train']))
train_dataset = dataset["train"]
shuffle_train_dataset = train_dataset.shuffle(seed=42)
test_dataset = shuffle_train_dataset.select(range(test_size))
new_train_dataset = shuffle_train_dataset.select(range(test_size, len(train_dataset)))

dataset = DatasetDict({
    "train": new_train_dataset,
    "validation": dataset["validation"],
    "test": test_dataset
})

In [ ]:

# train_size = len(dataset['train'])
# dataset['train'] = dataset['train'].select(range(train_size // 2))
# train_size = len(dataset['validation'])
# dataset['validation'] = dataset['validation'].select(range(train_size // 2))
# train_size = len(dataset['test'])
# dataset['test'] = dataset['test'].select(range(train_size // 2))

In [ ]:

files=next(os.walk(f'{path}saved_model/{file_to_train}'))[1]
results_eval = {file_model: [] for file_model in files}


In [6]:
counter=0
counter_2=0
for example in dataset['validation']:
    for answer in example["answers"]["text"]:
        if len(answer)>50:
            counter+=1
        else:
            counter_2+=1


In [26]:
counter

1334

In [27]:
counter_2

18103

In [7]:
metric = load("squad")


In [8]:
def preprocess_validation_examples(examples):
            questions = [q.strip() for q in examples["question"]]
            inputs = tokenizer(
                questions,
                examples["context"],
                max_length=128,
                truncation="only_second",
                stride=50,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length",
            )

            sample_map = inputs.pop("overflow_to_sample_mapping")
            example_ids = []

            for i in range(len(inputs["input_ids"])):
                sample_idx = sample_map[i]
                example_ids.append(examples["id"][sample_idx])

                sequence_ids = inputs.sequence_ids(i)
                offset = inputs["offset_mapping"][i]
                inputs["offset_mapping"][i] = [
                    o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
                ]

            inputs["example_id"] = example_ids
            return inputs
#https://huggingface.co/docs/transformers/en/tasks/question_answering
def preprocess_function(examples):
            questions = [q.strip() for q in examples["question"]]
            inputs = tokenizer(
                questions,
                examples["context"],
                max_length=128,
                truncation="only_second",
                return_offsets_mapping=True,
                padding="max_length",
            )

            offset_mapping = inputs.pop("offset_mapping")
            answers = examples["answers"]
            start_positions = []
            end_positions = []
            # try:
            for i, offset in enumerate(offset_mapping):
                    answer = answers[i]
                    start_char = answer["answer_start"][0]
                    end_char = answer["answer_start"][0] + len(answer["text"][0])
                    sequence_ids = inputs.sequence_ids(i)
                    sequence_ids = [seq_id if seq_id is not None else 0 for seq_id in inputs.sequence_ids(i)]
                    idx = 0
                    while sequence_ids[idx] != 1:
                        idx += 1
                    context_start = idx
                    while idx < len(sequence_ids) and sequence_ids[idx] == 1:
                        idx += 1
                    context_end = idx - 1
                    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
                        start_positions.append(0)
                        end_positions.append(0)
                    else:
                        idx = context_start
                        while idx <= context_end and offset[idx][0] <= start_char:
                            idx += 1
                        start_positions.append(idx - 1)
                        idx = context_end
                        while idx >= context_start and offset[idx][1] >= end_char:
                            idx -= 1
                        end_positions.append(idx + 1)   
            inputs["start_positions"] = start_positions
            inputs["end_positions"] = end_positions
            return inputs



def compute_metrics(start_logits, end_logits, features, examples):
            n_best = 20
            max_answer_length = 50
            example_to_features = collections.defaultdict(list)
            for idx, feature in enumerate(features):
                example_to_features[feature["example_id"]].append(idx)

            predicted_answers = []
            for example in tqdm(examples):
                example_id = example["id"]
                context = example["context"]
                answers = []

                for feature_index in example_to_features[example_id]:
                    start_logit = start_logits[feature_index]
                    end_logit = end_logits[feature_index]
                    offsets = features[feature_index]["offset_mapping"]

                    start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                    end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                    for start_index in start_indexes:
                        for end_index in end_indexes:
                            # Skip answers that are not fully in the context
                            if offsets[start_index] is None or offsets[end_index] is None:
                                continue
                            # Skip answers with a length that is either < 0 or > max_answer_length
                            if (
                                end_index < start_index
                                or end_index - start_index + 1 > max_answer_length
                            ):
                                continue
                            
                            answer = {
                                "text": context[offsets[start_index][0] : offsets[end_index][1]],
                                "logit_score": start_logit[start_index] + end_logit[end_index],
                            }
                            answers.append(answer)

                # Select the answer with the best score
                if len(answers) > 0:
                    best_answer = max(answers, key=lambda x: x["logit_score"])
                    predicted_answers.append(
                        {"id": example_id, "prediction_text": best_answer["text"]}
                    )
                else:
                    predicted_answers.append({"id": example_id, "prediction_text": ""})

            theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
            return metric.compute(predictions=predicted_answers, references=theoretical_answers)


In [9]:
learning_rate = [5e-5,3e-5,1e-5]
number_of_epochs = [3, 5,7]
combined_params = [{'learning_rate': lr, 'epoch': ep} for lr, ep in product(learning_rate, number_of_epochs)]

In [ ]:
torch.cuda.empty_cache()
gc.collect()
for parameters in combined_params:
    learning_rate=parameters['learning_rate']
    number_of_epochs=parameters['epoch']
    for file_to_load in files:
        results_eval = {} 
        save_path = f"{path}saved_model/{file_to_train}/{file_to_load}"
        tokenizer = AutoTokenizer.from_pretrained(save_path)
        qa_model=AutoModelForQuestionAnswering.from_pretrained(save_path)
        torch.cuda.empty_cache()
        gc.collect()
        validation_dataset = dataset["validation"].map(preprocess_validation_examples,batched=True,remove_columns=dataset["validation"].column_names)
        tokenized_dataset_train =  dataset["train"].map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
        tokenized_dataset_train = tokenized_dataset_train.select(range(len(tokenized_dataset_train)//2))
        tokenized_dataset_test =  dataset["test"].map(preprocess_function, batched=True, remove_columns=dataset["test"].column_names)
        torch.cuda.empty_cache()
        gc.collect()
        time.sleep(15)
        print('LOL')
        training_args = TrainingArguments(
            output_dir=f"{path}saved_model/{file_to_train}/{file_to_load}/QA/results",
            evaluation_strategy="epoch",
            learning_rate=learning_rate,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=number_of_epochs,
            weight_decay=0.01,
            save_strategy="epoch",
            logging_dir=f"{path}saved_model/{file_to_train}/{file_to_load}/QA/logs",
            push_to_hub=False
        )
        trainer = Trainer(
            model=qa_model,
            args=training_args,
            train_dataset=tokenized_dataset_train,
            eval_dataset=tokenized_dataset_test,
            processing_class=tokenizer,
            data_collator=default_data_collator,
        )
        trainer.train()
        torch.cuda.empty_cache()
        gc.collect()
        predictions, _, _ = trainer.predict(validation_dataset)
        start_logits, end_logits = predictions
        result=compute_metrics(start_logits, end_logits, validation_dataset, dataset["validation"])
        result['parameters']=[learning_rate,number_of_epochs]
        torch.cuda.empty_cache()
        gc.collect()
        results_eval[file_to_load]=result
        with open(f'{path}results_evaluation/{file_to_train}/output_QA.json', 'a') as f:
            json.dump(results_eval, f)
            f.write(',\n')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /data/lubosk/diploma_thesis/saved_model/strict_model_individual_datasets/randomly_selected_subs and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5729 [00:00<?, ? examples/s]

Map:   0%|          | 0/74635 [00:00<?, ? examples/s]

Map:   0%|          | 0/8292 [00:00<?, ? examples/s]

LOL


...

/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:34: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,4.283300,4.172488
2,4.143300,4.080168
3,4.082000,4.055958


/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:34: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimens

  0%|          | 0/5729 [00:00<?, ?it/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /data/lubosk/diploma_thesis/saved_model/strict_model_individual_datasets/randomly_selected_wiki_full and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
